In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.model_selection import train_test_split

In [3]:
random_state = 100

In [4]:
data = pd.read_csv('Bias_correction_ucl.csv')
data['Date'] = pd.to_datetime(data['Date'])

data = data[data['Next_Tmax'].notnull() & data['Next_Tmin'].notnull()]
data = data[data['Date'].notnull() & data['station'].notnull()]
data = data[data['LDAPS_Tmax_lapse'].notnull() & data['LDAPS_Tmin_lapse'].notnull()]

y_Tmax = data['Next_Tmax']
y_Tmin = data['Next_Tmin']
X = data.drop(['Next_Tmax', 'Next_Tmin'], axis='columns')
print(X.shape, y_Tmax.shape, y_Tmin.shape)
X.head()

(7648, 23) (7648,) (7648,)


,station,Date,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,...,LDAPS_CC4,LDAPS_PPT1,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation
0,1.0,2013-06-30,28.7,21.4,58.255688,91.116364,28.074101,23.006936,6.818887,69.451805,...,0.130928,0.0,0.0,0.0,0.0,37.6046,126.991,212.3350,2.7850,5992.895996
1,2.0,2013-06-30,31.9,21.6,52.263397,90.604721,29.850689,24.035009,5.691890,51.937448,...,0.127727,0.0,0.0,0.0,0.0,37.6046,127.032,44.7624,0.5141,5869.312500
2,3.0,2013-06-30,31.6,23.3,48.690479,83.973587,30.091292,24.565633,6.138224,20.573050,...,0.142125,0.0,0.0,0.0,0.0,37.5776,127.058,33.3068,0.2661,5863.555664
3,4.0,2013-06-30,32.0,23.4,58.239788,96.483688,29.704629,23.326177,5.650050,65.727144,...,0.134249,0.0,0.0,0.0,0.0,37.6450,127.022,45.7160,2.5348,5856.964844
4,5.0,2013-06-30,31.4,21.9,56.174095,90.155128,29.113934,23.486480,5.735004,107.965535,...,0.170021,0.0,0.0,0.0,0.0,37.5507,127.135,35.0380,0.5055,5859.552246


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y_Tmax, test_size=0.2, shuffle=True, random_state=random_state)
# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=100)

In [6]:
class MissingDataRowsDropper(BaseEstimator, TransformerMixin):
    
    def __init__(self, cols):
        if cols is None:
            cols = []
        self.cols = cols
        self.predicate = None
    
    def fit(self, X, y=None):
        if any([v not in X.columns for v in self.cols]):
            raise Exception("Wrong column name provided")
        self.predicate = X[self.cols].notnull().all(axis='columns')
        return self
    
    def transform(self, X, y=None):
        if self.predicate is None:
            raise Exception("Have not been fed before transformation")
        if y:
            return X[self.predicate], y[self.predicate]
        return X[self.predicate]
        

In [7]:
preprocessing = make_pipeline(
#     MissingDataRowsDropper(cols=['station', 'Date']), 
    make_column_transformer(
         (SimpleImputer(strategy='median'), ['Present_Tmax', 'Present_Tmin', 'LDAPS_RHmin',
       'LDAPS_RHmax', 'LDAPS_Tmax_lapse', 'LDAPS_Tmin_lapse', 'LDAPS_WS',
       'LDAPS_LH', 'LDAPS_CC1', 'LDAPS_CC2', 'LDAPS_CC3', 'LDAPS_CC4',
       'LDAPS_PPT1', 'LDAPS_PPT2', 'LDAPS_PPT3', 'LDAPS_PPT4'])
    ),
    StandardScaler()
)

In [8]:
class BaseModel(BaseEstimator, ClassifierMixin):
    
    def __init__(self, to_predict):
        if to_predict not in ['Next_Tmax', 'Next_Tmin']:
            raise Exception('Unknown prediction type')
        self.to_predict = to_predict
    
    def fit(self, X, y):
        return self
    
    def predict(self, X):
        if self.to_predict == 'Next_Tmax':
            return X['LDAPS_Tmax_lapse']
        else:
            return X['LDAPS_Tmin_lapse']


In [26]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [10]:
k_fold = KFold(n_splits=5, shuffle=True, random_state=random_state)
models = {}

In [11]:
model = Pipeline([
    ('preprocessing', preprocessing), 
    ('regressor', LinearRegression())
])

model.fit(X_train, y_train)
models['Linear'] = model

In [12]:
model = Pipeline([
    ('preprocessing', preprocessing), 
    ('regressor', Ridge())
])
params = {
    'regressor__alpha': [0.1, 1, 10, 100, 1000]
}

grid = GridSearchCV(
    estimator=model, 
    param_grid=params,
    n_jobs=-1,
    verbose=3,
    cv=k_fold
)
grid.fit(X_train, y_train)
print(grid.best_params_)
models['Ridge'] = grid.best_estimator_

Fitting 5 folds for each of 5 candidates, totalling 25 fits
{'regressor__alpha': 1}


In [13]:
model = Pipeline([
    ('preprocessing', preprocessing), 
    ('regressor', Lasso())
])
params = {
    'regressor__alpha': [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
}

grid = GridSearchCV(
    estimator=model, 
    param_grid=params,
    n_jobs=-1,
    verbose=3,
    cv=k_fold
)
grid.fit(X_train, y_train)
print(grid.best_params_)
models['Lasso'] = grid.best_estimator_

Fitting 5 folds for each of 7 candidates, totalling 35 fits
{'regressor__alpha': 0.001}


In [37]:
model = Pipeline([
    ('preprocessing', preprocessing), 
    ('regressor', SVR(kernel='rbf'))
])
params = {
#     'regressor__kernel': ['rbf'],
    'regressor__C': [1e-1, 1e1, 1e2]
}

grid = GridSearchCV(
    estimator=model, 
    param_grid=params,
    n_jobs=-1,
    verbose=3,
    cv=k_fold
)
grid.fit(X_train, y_train)
print(grid.best_params_)
models['SVR rbf'] = grid.best_estimator_

Fitting 5 folds for each of 3 candidates, totalling 15 fits
{'regressor__C': 100.0}


In [38]:
model = Pipeline([
    ('preprocessing', preprocessing), 
    ('regressor', SVR(kernel='linear'))
])
params = {
    'regressor__C': [1e-3, 1e-2, 1e-1, 1e1, 1e2]
}

grid = GridSearchCV(
    estimator=model, 
    param_grid=params,
    n_jobs=-1,
    verbose=3,
    cv=k_fold
)
grid.fit(X_train, y_train)
print(grid.best_params_)
models['SVR linear'] = grid.best_estimator_

Fitting 5 folds for each of 5 candidates, totalling 25 fits
{'regressor__C': 100.0}


In [33]:
model = Pipeline([
    ('preprocessing', preprocessing), 
    ('regressor', RandomForestRegressor())
])
params = {
    'regressor__max_depth': [4, 6, 8, 16, 32, 64],
    'regressor__max_features': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'regressor__max_leaf_nodes': [8, 16, 32, 64, 128]
}

grid = RandomizedSearchCV(
    estimator=model, 
    param_distributions=params,
    n_jobs=-1,
    verbose=3,
    cv=k_fold,
    n_iter=100
)
grid.fit(X_train, y_train)
print(grid.best_params_)
models['Random Forest'] = grid.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'regressor__max_leaf_nodes': 128, 'regressor__max_features': 0.4, 'regressor__max_depth': 16}


# Models evaluation

In [39]:
from sklearn.metrics import mean_squared_error

In [40]:
models['base_model'] = BaseModel('Next_Tmax')

In [41]:
for name, model in models.items():
    print(name)
    print(f'mean_squared_error: {mean_squared_error(model.predict(X_test), y_test)}')

Linear
mean_squared_error: 2.2798476266027086
Ridge
mean_squared_error: 2.2798676964794
Lasso
mean_squared_error: 2.2800170896241574
base_model
mean_squared_error: 3.6321580125711477
SVR
mean_squared_error: 2.2950740683093915
Random Forest
mean_squared_error: 1.3955820748043255
SVR rbf
mean_squared_error: 0.823176584172425
SVR linear
mean_squared_error: 2.2950740683093915
